

```
define Professor Dr._Akbari
define TA Mr._Malekzade 
define studentID 9513004  
define studentName Ardestani

```



In [1]:
# Authentication and Configuration
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from google.colab import drive
import warnings
drive.mount('/gdrive', force_remount=True)
!ls
import os 
os.getcwd()
%cd /gdrive/MyDrive/NLPcourse/Scraping_Crawling
!ls

Mounted at /gdrive
adc.json  sample_data
/gdrive/MyDrive/NLPcourse/Scraping_Crawling
checkpoints.csv  data.csv  FinalData.csv  pictures


# **Main part (Scraping Houzz.com)**

## First try using bs4 (Failed)

 **At the end of the first try we store all images with their descriptions in the pictures folder on our local direcrtory 
but the poblem of this part is bs4 is not suitable and it's slow** 

Importin libraries 

In [2]:
!pip3 install bs4

In [3]:
from bs4 import BeautifulSoup
import requests
root = 'https://www.houzz.com/products/'
URL = root + 'outdoor-coffee-tables'
URL1 = root + 'outdoor-coffee-tables/p/36?oq='
URL2 = root + 'outdoor-coffee-tables/p/72?oq='
URL3 = root + 'outdoor-coffee-tables/p/108?oq='
page = requests.get(URL)
page1 = requests.get(URL1)
page2 = requests.get(URL2)
page3 = requests.get(URL3)

s = BeautifulSoup(page.content, 'html.parser')
s1 = BeautifulSoup(page1.content, 'html.parser')
s2 = BeautifulSoup(page2.content, 'html.parser')
s3 = BeautifulSoup(page3.content, 'html.parser')

images = s.find_all(class_='hz-product-card__link')
images.extend(s1.find_all(class_='hz-product-card__link'))
images.extend(s2.find_all(class_='hz-product-card__link'))
images.extend(s3.find_all(class_='hz-product-card__link'))
imagesAddress = [image.get("href") for image in images]

folder = './pictures/'
counter  = 0
for src in imagesAddress[:10]:
    if src is not None:
        page_temp = requests.get(src)
        print(page_temp)
        soup_temp = BeautifulSoup(page_temp.content, 'html.parser')
        results = soup_temp.find_all(class_='view-product-title')
        if results:
            print(results[0].text)
            results_temp = soup_temp.find_all(class_='alt-images__thumb')
            imgs = [result.find('img') for result in results_temp]
            images = [item.get('src') for item in imgs]
            for image in images:
                file_name = 'pic' + str(counter) + '.jpg'
                counter += 1
                with open(folder+file_name, 'wb') as handle:
                    try:
                        response = requests.get(image, stream=True)
                        if not response.ok:
                            print(response)
                        for block in response.iter_content(1024):
                            if not block:
                                break#if it's not block we beack this search 
                            handle.write(block)
                    except:
                        pass


<Response [200]>
Grand Terrace 42" Round Chat Table
<Response [200]>
Brazil Suar Wood Unique Slab Live Edge Coffee Table
<Response [200]>
Teak Wood Coffee Table With Wheels
<Response [200]>
Fairmont Storage Coffee Table Vanilla Creme
<Response [200]>
vidaXL Balcony Table Mosaic Hanging Semi-circular Black White Patio Side Desk
<Response [200]>
Teak Wood Root Coffee Table With Glass Top, 55"


## second try using scrapy ( Failed in running time saving)


***this try perfectly works and it's relatively fast But it's weekness is: we save data at the end of scraping ***

In [4]:
!pip3 install scrapy

     |████████████████████████████████| 245kB 5.4MB/s 
     |████████████████████████████████| 245kB 7.2MB/s 
     |████████████████████████████████| 2.6MB 8.4MB/s 
     |████████████████████████████████| 3.1MB 16.3MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 3.2MB 47.3MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7766 sha256=2f801d8a32b9e9cda75899ef6e3f0c6b580b88559acf46d6f67ce944147261ec
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=cf29f222b1e599d63afe6c1067a8d4cb1184424a101c1749b78d56089c6ea45f
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego PyDi

In [5]:
import scrapy
from scrapy.crawler import CrawlerProcess
import csv

ListOfImages = []
NameOfproduct = []
tags = [] # product description tags
urls = []
FinalResults = [] # we store this list in a csv file at the end of the process
domain = 'https://www.houzz.com'

''' change this parameters if you would like '''
NumberOfPictures  = 2 # number of pic that we wanna scrape
NumberOfPages = 1 # number of pages that we wanaa go through

class ItemOfPriducts(scrapy.Item):
    Source = scrapy.Field()
    alt = scrapy.Field()
    Description = scrapy.Field()
class ScrapySpider(scrapy.Spider):
    name = "product"
    start_urls = ['https://www.houzz.com/products/chairs','https://www.houzz.com/products/sofas','https://www.houzz.com/products/beds','https://www.houzz.com/products/tables',]

    def parse(self, response):
        Urls = response.xpath('//a[contains(@class,"hz-pagination-link")]/@href').extract()
        print('HIII ', len(Urls))
        for link in Urls[:NumberOfPages]:
            print('Urls: ', domain + link)
            yield scrapy.Request(domain + link, callback=self.parse_page)

    def parse_page(self, response):
        Urls = response.xpath('//a[@class="hz-product-card__link"]/@href').extract()
        for link in Urls:
            urls.append(link)
            yield scrapy.Request(link, callback=self.parse_attr)

    def parse_attr(self, response):
        item = ItemOfPriducts()
        item["Source"] = response.xpath("//div[contains(@class, 'alt-images__thumb')]//img/@src").extract()
        item["alt"] = response.xpath("//div[contains(@class, 'alt-images__thumb')]//img/@alt").extract()
        item["Description"] = response.xpath("//li[contains(@class, 'product-keywords__word')]//text()").extract()
        ListOfImages.append(item["Source"][:NumberOfPictures ])
        NameOfproduct.append(item["alt"][0])
        tags.append(item["Description"])
        FinalResults.append([NameOfproduct[-1],tags[-1],ListOfImages[-1][0], ListOfImages[-1][1]])
        return item

# run spider
process = CrawlerProcess()
process.crawl(ScrapySpider)
process.start()

with open(os.path.join(os.getcwd(),'data.csv'), 'w') as f:
  write = csv.writer(f)
  write.writerow(['NameOfproduct', 'description tages', 'pic1', 'pic2'])
  write.writerows(FinalResults)

2021-01-04 18:15:48 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-01-04 18:15:48 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Oct  8 2020, 12:12:24) - [GCC 8.4.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
2021-01-04 18:15:48 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-01-04 18:15:48 [scrapy.crawler] INFO: Overridden settings:
{}
2021-01-04 18:15:48 [scrapy.extensions.telnet] INFO: Telnet Password: fa6e72a71b663d5c
2021-01-04 18:15:48 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-01-04 18:15:49 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.download

HIII  10
Urls:  https://www.houzz.com/products/beds
HIII  10
Urls:  https://www.houzz.com/products/query/tables


2021-01-04 18:15:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.houzz.com/products/chairs> (referer: None)


HIII  10
Urls:  https://www.houzz.com/products/sofas
HIII  10
Urls:  https://www.houzz.com/products/chairs


2021-01-04 18:15:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.houzz.com/products/query/tables> (referer: https://www.houzz.com/products/tables)
2021-01-04 18:15:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.houzz.com/products/beds> (referer: https://www.houzz.com/products/beds)
2021-01-04 18:15:51 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.houzz.com/products/30-square-black-laminate-table-top-with-22-x22-table-height-base-prvw-vr~160671573> from <GET https://www.houzz.com/products/30-square-black-laminate-table-top-with-22x22-table-height-base-prvw-vr~160671573>
2021-01-04 18:15:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.houzz.com/products/chairs> (referer: https://www.houzz.com/products/chairs)
2021-01-04 18:15:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.houzz.com/products/sofas> (referer: https://www.houzz.com/products/sofas)
2021-01-04 18:15:52 [scrapy.core.engine] DEBUG: 

## Third try Using scrapy and additinal funcs (successful)
** The final try works. In this part can store the data at running time so that we could manage internet disconnection and similar incidents **

In [4]:
!pip3 install scrapy
import scrapy
from scrapy.crawler import CrawlerProcess
import csv

''' set these parameters if you would like'''
WebsiteRootAddress = 'https://www.houzz.com'
NumberOfPictures = 2 # number of pictures that we wanna scrape
imagesLikns = [] 
nameOfPriducts = []
descriptionTags = []
urls = []
types = []

''' change these parameters if you would like ''' 
 # name of categories that we wanna scrape 
categories = ['chairs', 'sofas', 'beds', 'tables']
# startingpage shows from which page we wanna start scraping 
StartingPage = {'chairs': 0, 'sofas': 0, 'beds': 0, 'tables': 0} 
# Endingpage shows from which page we wanna start scraping
EndingPage = {'chairs': 1, 'sofas': 1, 'beds': 1, 'tables': 1} 
# name of the file for saving data in running time
nameOfcsv = 'FinalData.csv' 

Adding = 0  # At to current existing file on the computer
Interruption = 0 # whether we've ecountered any interruption or not
PageAfterInterruption = 0  # number of page after interruption

TotalNumberOfCategories = {content : 0 for content in categories}
# creating the output file and headlines 
if not (Adding or Interruption):
  with open(nameOfcsv,'w') as fd:
      csv_writer = csv.writer(fd)
      csv_writer.writerow(['nameOfPriducts', 'described as', 'image 1', 'image 2', 'url'])

# Interruptions management 
if Interruption:
  with open('checkpoints.csv', "r") as f:
    reader = csv.reader(f)
    lines = []
    for i, line in enumerate(reader):
        # print('line[{}] = {}'.format(i, line))
        lines.append(line)
    categories = lines[0]
    TotalNumberOfCategories = {categories[j]: int(lines[1][j]) for j in range(len(categories))}
    if not PageAfterInterruption:
      EndingPage = {categories[j]: int(lines[2][j]) for j in range(len(categories))}
    nameOfcsv = lines[3][0]
    
## read the following notes if you wanna get the gist of this part 
''' now with the following variables we can say whether we have intruption or we are at the first run'''
''' Let me briefly say how we manage interruptions .'''
''' We have another file in our computer's dirve named chekcpoints and we store the number of last page that we have scraped'''
''' so we the following variables you can easily tell the scraper that from which pages we should start'''
''' if you have encountered any interruption set _interruption_ , _Adding_ vars equal to 1 ''' 

class Items(scrapy.Item):
    src = scrapy.Field()
    alt = scrapy.Field()
    describe = scrapy.Field()
class ScrapySpider(scrapy.Spider):
    name = "product"
    content_ptr = 0
    start_urls = ['https://www.houzz.com/products',]

    def parse(self, response): 
        if TotalNumberOfCategories[categories[self.content_ptr]] >= EndingPage[categories[self.content_ptr]]: # it means we have finished scraping for corresponding category
          if self.content_ptr >= len(categories) - 1:
            with open('checkpoints.csv','w') as fd: # updating checkpoints 
                csv_writer = csv.writer(fd)
                csv_writer.writerows([categories, TotalNumberOfCategories.values(), EndingPage.values()])
                csv_writer.writerow([nameOfcsv])
            return
          self.content_ptr += 1
          yield scrapy.Request(self.start_urls[0] + '/' + categories[self.content_ptr], callback=self.parse)
        else:
          if response.url == self.start_urls[0]: # first time in the function! going to the real urls
            yield scrapy.Request(self.start_urls[0] + '/' + categories[self.content_ptr], callback=self.parse)
          else:
            next_link = response.xpath('//a[contains(@class,"hz-pagination-link--next")]/@href').extract()
            if TotalNumberOfCategories[categories[self.content_ptr]] < StartingPage[categories[self.content_ptr]]: # moving to the start page
              TotalNumberOfCategories[categories[self.content_ptr]] += 1
              with open('checkpoints.csv','w') as fd: # checkpoint
                csv_writer = csv.writer(fd)
                csv_writer.writerows([categories, TotalNumberOfCategories.values(), EndingPage.values()])
                csv_writer.writerow([nameOfcsv])
              yield scrapy.Request(WebsiteRootAddress + next_link[0], callback=self.parse)
            else: # looking for images
              links = response.xpath('//a[@class="hz-product-card__link"]/@href').extract()
              for link in links:
                  urls.append(link)
                  types.append(categories[self.content_ptr])
                  yield scrapy.Request(link, callback=self.attributeParser)
              with open('checkpoints.csv','w') as fd: # checkpoint
                csv_writer = csv.writer(fd)
                csv_writer.writerows([categories, TotalNumberOfCategories.values(), EndingPage.values()])
                csv_writer.writerow([nameOfcsv])
              yield scrapy.Request(WebsiteRootAddress + next_link[0], callback=self.parse)
              TotalNumberOfCategories[categories[self.content_ptr]] += 1

    def attributeParser(self, response):
        item = Items()
        item["src"] = response.xpath("//div[contains(@class, 'alt-images__thumb')]//img/@src").extract()
        item["alt"] = response.xpath("//div[contains(@class, 'alt-images__thumb')]//img/@alt").extract()
        item["describe"] = response.xpath("//li[contains(@class, 'product-keywords__word')]//text()").extract()
        imagesLikns.append(item["src"][:NumberOfPictures]) # we only store 2 number of images (we can change this with parameters)
        if item["alt"]:
          nameOfPriducts.append(item["alt"][0])
        else: 
          nameOfPriducts.append('')
        descriptionTags.append(item["describe"]) # updaing descriptionTags
        csv_results = [nameOfPriducts[-1], descriptionTags[-1], imagesLikns[-1][0], imagesLikns[-1][1], response.url] #adding data in running time into the output
        with open(nameOfcsv,'a+') as fd:
          csv_writer = csv.writer(fd)
          csv_writer.writerow(csv_results)
        return item

# Driver code for crawling 
process = CrawlerProcess()
process.crawl(ScrapySpider)
process.start()

     |████████████████████████████████| 245kB 5.7MB/s 
     |████████████████████████████████| 245kB 12.6MB/s 
     |████████████████████████████████| 2.6MB 15.8MB/s 
     |████████████████████████████████| 3.1MB 38.3MB/s 
     |████████████████████████████████| 3.2MB 41.1MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=551d430d60c02194ef90040b9f699b0f62ead8738392c539a18596854fecb01c
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7766 sha256=71ba5d018ed125cb09164caf04d3259f58d2cc2d97e62d20a52c58481d64441c
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatch

2021-01-04 18:19:22 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-01-04 18:19:22 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Oct  8 2020, 12:12:24) - [GCC 8.4.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
2021-01-04 18:19:22 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-01-04 18:19:22 [scrapy.crawler] INFO: Overridden settings:
{}
2021-01-04 18:19:22 [scrapy.extensions.telnet] INFO: Telnet Password: cce36e1cf11dfa87
2021-01-04 18:19:22 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-01-04 18:19:22 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.download

# **Bonus part** (Blogfa And IP rotation)

## BlogFu

In [5]:
!pip3 install bs4

In [6]:
from bs4 import BeautifulSoup
import requests
import csv
import re 

#we only have 5 pages in /update/ directory 
URLs = ['http://blogfa.com/updated/?p=1', 
        'http://blogfa.com/updated/?p=2',
        'http://blogfa.com/updated/?p=3',
        'http://blogfa.com/updated/?p=4',
        'http://blogfa.com/updated/?p=5'] 

def soupMaker(urls):
  lenght = len(urls)
  listOfSoups = []
  for i in range(lenght):
    source = requests.get(urls[i]).text
    soup = BeautifulSoup(source, 'lxml')
    listOfSoups.append(soup)
  return listOfSoups 
def linkExtracter(soups):
  listOfLinks = []
  for soup in soups:
    target = soup.find_all('div', class_='updatelist')
    tmpList = []
    for urlTag in target:
      #honestly I've a vague idea about this line.It works bya miracle:)
      tmpList = urlTag.find_all('li') 
    for i in range(len(tmpList)):
      listOfLinks.append(tmpList[i].a['href'])
  return listOfLinks 

In [7]:
listOfRecentURLs = linkExtracter(soupMaker(URLs))
print(listOfRecentURLs)

2021-01-04 18:20:23 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): blogfa.com:80
2021-01-04 18:20:23 [urllib3.connectionpool] DEBUG: http://blogfa.com:80 "GET /updated/?p=1 HTTP/1.1" 200 None
2021-01-04 18:20:23 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): blogfa.com:80
2021-01-04 18:20:23 [urllib3.connectionpool] DEBUG: http://blogfa.com:80 "GET /updated/?p=2 HTTP/1.1" 200 None
2021-01-04 18:20:23 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): blogfa.com:80
2021-01-04 18:20:24 [urllib3.connectionpool] DEBUG: http://blogfa.com:80 "GET /updated/?p=3 HTTP/1.1" 200 None
2021-01-04 18:20:24 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): blogfa.com:80
2021-01-04 18:20:24 [urllib3.connectionpool] DEBUG: http://blogfa.com:80 "GET /updated/?p=4 HTTP/1.1" 200 None
2021-01-04 18:20:24 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): blogfa.com:80
2021-01-04 18:20:24 [urllib3.connectionpool] DEBUG: http

['http://kar20kanaf.blogfa.com/post/8', 'http://azar999.blogfa.com/post/5', 'http://dorsaramezani88.blogfa.com/post/3', 'http://afarinafshari.blogfa.com/post/5', 'http://zahrabti.blogfa.com/post/14', 'http://mm2410.blogfa.com/post/3', 'http://barayehana.blogfa.com/post/5', 'http://mcdonalds.blogfa.com/post/7', 'http://armankousha.blogfa.com/post/3', 'http://jafarrezaiee.blogfa.com/post/265', 'http://animallaw.blogfa.com/post/4', 'http://razebandegi99.blogfa.com/post/20', 'http://bagh2020ketab.blogfa.com/post/46', 'http://play16.blogfa.com/post/11', 'http://skinkaraj.blogfa.com/post/6', 'http://garmiranservice.blogfa.com/post/31', 'http://yagub62.blogfa.com/post/158', 'http://tinuviiel.blogfa.com/post/16', 'http://mfvsah.blogfa.com/post/388', 'http://miranda.blogfa.com/post/4', 'http://sipadd.blogfa.com/post/3', 'http://ghazalatena.blogfa.com/post/3', 'http://bodobazi.blogfa.com/post/6', 'http://jahanonlineblog.blogfa.com/post/59', 'http://zeynabmoosavie80.blogfa.com/post/7', 'http://ar

In [8]:
# So now that we have fetched all recent blogs URLs we can find recent post by decreasing the last number of the address by one
## But important thing is notice that some post might be removed 
### SO if a post is not exist ( 404 not found error ) we just skip that post 

Last_10_Posts_of_all_blogs = {} # links of the last ten posts for each blog 

""" change this parameter if you would like """
kk = 2 # number of blogs that we wanna scrape 
## note if you wanna scrape all blogs put kk = len(listOfRecentURLs)

for url_index, URLaddress in enumerate(listOfRecentURLs[:kk]):
    temporary = re.sub(r"(.*post\/)", "", URLaddress)
    # temporary2 is the main address of a blog For instanse if instanse it's like "nameofBlog.blogfa.com/nunmberOfPost"
    temporary2 = re.sub(r"(post.*)", "", URLaddress) 
    # temporary3 is the number of the blog's last post  
    temporary3 = re.sub(r"(\/.*)", "", temporary)
    # Assume the last post's number is X , we decrease X by one each time for scraping them
    last_posts_urls = [temporary2 + "post"+"/"+ str(int(temporary3)-index) for index in range(10) if int(temporary3)-index>0]
    # now in this loop we gonna scrape of last posts in each blogs 
    for index, post in enumerate(last_posts_urls):
        if requests.get(post).status_code == 404:
            last_posts_urls.remove(post)
    Last_10_Posts_of_all_blogs[url_index] = last_posts_urls

print(Last_10_Posts_of_all_blogs)

2021-01-04 18:20:28 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): kar20kanaf.blogfa.com:80
2021-01-04 18:20:28 [urllib3.connectionpool] DEBUG: http://kar20kanaf.blogfa.com:80 "GET /post/8 HTTP/1.1" 200 None
2021-01-04 18:20:28 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): kar20kanaf.blogfa.com:80
2021-01-04 18:20:28 [urllib3.connectionpool] DEBUG: http://kar20kanaf.blogfa.com:80 "GET /post/7 HTTP/1.1" 200 None
2021-01-04 18:20:28 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): kar20kanaf.blogfa.com:80
2021-01-04 18:20:28 [urllib3.connectionpool] DEBUG: http://kar20kanaf.blogfa.com:80 "GET /post/6 HTTP/1.1" 200 None
2021-01-04 18:20:28 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): kar20kanaf.blogfa.com:80
2021-01-04 18:20:28 [urllib3.connectionpool] DEBUG: http://kar20kanaf.blogfa.com:80 "GET /post/5 HTTP/1.1" 200 None
2021-01-04 18:20:28 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): kar20k

{0: ['http://kar20kanaf.blogfa.com/post/8', 'http://kar20kanaf.blogfa.com/post/7', 'http://kar20kanaf.blogfa.com/post/6', 'http://kar20kanaf.blogfa.com/post/5', 'http://kar20kanaf.blogfa.com/post/4', 'http://kar20kanaf.blogfa.com/post/3', 'http://kar20kanaf.blogfa.com/post/2', 'http://kar20kanaf.blogfa.com/post/1'], 1: ['http://azar999.blogfa.com/post/5', 'http://azar999.blogfa.com/post/4', 'http://azar999.blogfa.com/post/3', 'http://azar999.blogfa.com/post/2', 'http://azar999.blogfa.com/post/1']}


## IP rotation

In [9]:
!pip3 install fake-useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=b768b81fd961d31adbcdd1c106465fb17e99d797dfa246c65b4226b544173005
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [10]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random

In [11]:
ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]
# Retrieve latest proxies
proxies_req = Request('https://www.sslproxies.org/')
proxies_req.add_header('User-Agent', ua.random)
proxies_doc = urlopen(proxies_req).read().decode('utf8')
soup = BeautifulSoup(proxies_doc, 'html.parser')
proxies_table = soup.find(id='proxylisttable')
# Save proxies in the array
for row in proxies_table.tbody.find_all('tr'):
  proxies.append({
    'ip':   row.find_all('td')[0].string,
    'port': row.find_all('td')[1].string
  })

In [12]:
print(proxies) #, proxies_doc, proxies_req)

[{'ip': '139.162.100.41', 'port': '8080'}, {'ip': '31.172.105.144', 'port': '8080'}, {'ip': '95.105.139.253', 'port': '3128'}, {'ip': '124.158.88.56', 'port': '54555'}, {'ip': '91.108.137.197', 'port': '8080'}, {'ip': '175.100.103.170', 'port': '55443'}, {'ip': '103.206.102.240', 'port': '8080'}, {'ip': '170.79.85.224', 'port': '55443'}, {'ip': '103.106.219.102', 'port': '3128'}, {'ip': '128.14.163.94', 'port': '3128'}, {'ip': '103.83.116.202', 'port': '55443'}, {'ip': '162.127.140.112', 'port': '8080'}, {'ip': '159.203.82.173', 'port': '3128'}, {'ip': '146.59.199.43', 'port': '80'}, {'ip': '84.247.51.123', 'port': '3128'}, {'ip': '208.80.28.208', 'port': '8080'}, {'ip': '81.12.30.83', 'port': '8080'}, {'ip': '103.253.27.108', 'port': '80'}, {'ip': '60.246.7.4', 'port': '8080'}, {'ip': '144.217.101.245', 'port': '3129'}, {'ip': '187.62.191.3', 'port': '61456'}, {'ip': '109.238.222.5', 'port': '40387'}, {'ip': '201.217.4.101', 'port': '53281'}, {'ip': '190.171.180.162', 'port': '50846'}

In [13]:
## with help of Fake_useragent and sslproxies we change ip and we clear our cookies automatically at each request 
## So that we can avoid banning wiht this libbrary 

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random

         ### Set following parameters if you would like ###   
''' K = Number of requests with one proxy   '''
K = 2  
''' N = Total numbet of requests ''' 
N = 5 
ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]


# Main function
def main():
  # Retrieve latest proxies
  proxies_req = Request('https://www.sslproxies.org/')
  proxies_req.add_header('User-Agent', ua.random)
  proxies_doc = urlopen(proxies_req).read().decode('utf8')

  soup = BeautifulSoup(proxies_doc, 'html.parser')
  proxies_table = soup.find(id='proxylisttable')

  # Save proxies in the array
  for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

  # Choose a random proxy
  proxy_index = random_proxy()
  proxy = proxies[proxy_index]

  for n in range(1, N+1):
    req = Request('http://blogfa.com/updated/')
    req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

    # Every K requests, generate a new proxy
    if n % K == 0:
      proxy_index = random_proxy()
      proxy = proxies[proxy_index]

    # Make the call
    try:
      my_ip = urlopen(req).read().decode('utf8')
      print('#' + str(n) + ': ' + my_ip)
    except: # If error, delete this proxy and find another one
      del proxies[proxy_index]
      print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
      proxy_index = random_proxy()
      proxy = proxies[proxy_index]

# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
  return random.randint(0, len(proxies) - 1)

if __name__ == '__main__':
  main()


   



#1: 
<!DOCTYPE html>
<html>
	<head>
		<title>BLOGFA :: Free Persian Weblog Service :: وبلاگ فارسی </title>
		<meta http-equiv="Content-Type" content="text/html; charset=utf-8" >
        <meta name="viewport" content="width=device-width, initial-scale=1">
		<meta name="description"  content="BlogFa :: بلاگفا یک ابزار رایگان برای ساخت و مدیریت وبلاگ است" >
		<link href="/static/css/int.5.css" type="text/css" rel="stylesheet">
<!--[if lt IE 8]>
<style>
#footer li{display:inline;padding-left:5px;padding-right:5px;}
</style>
<![endif]-->
<script>
function togglemenu(){var e=n("menu");function n(e){return document.getElementById(e)}"menuhide"===e.className?(e.className="menushow",n("menutoggle").innerText="✖"):(e.className="menuhide",n("menutoggle").innerText="☰")}
</script>


    			<style>
			.ull {color: red;font-size:13px;margin-bottom:15px;line-height:25px;padding:20px;}
			.ull .sm{color:gray;display:block;margin:0px;padding:0px;font-size:12px;}
			.ull .bt{color:green;text-direction:r

# A little creativity (Extra work for having fun with python :) )

## Part one 
WE are going to scrape text speeches of some american stand-up comedians.

We gather our data form scrapsfromtheloft.com Website ( It's like IMDB ).  

and also at the end we're going to use **Pickle** library for sroing data.

(Pickle is an efficient library that we can pickeling(stroing) wide range of data types even python's objects  like, Dictionary ,or list, or an object of a class which is defined in our code.)

**Objective** : In future,I wanna use this data to show that comedy is highly based on The **paradox**. It's a little tricky, but I will do it with DL methods asap.

In [14]:
# Importing Libraries 
import requests
from bs4 import BeautifulSoup
import pickle

# Scraper func
def transcriptMaker(url):
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    text = [p.text for p in soup.find(class_="elementor-widget-theme-post-content").find_all('p')]
    print(url)
    return text

# URLs of transcripts in scope
domain = 'http://scrapsfromtheloft.com/'
urls = ['2017/05/06/louis-ck-oh-my-god-full-transcript/',
        '2017/04/11/dave-chappelle-age-spin-2017-full-transcript/',
        '2018/03/15/ricky-gervais-humanity-transcript/',
        '2017/08/07/bo-burnham-2013-full-transcript/',
        '2017/05/24/bill-burr-im-sorry-feel-way-2014-full-transcript/',
        '2017/04/21/jim-jefferies-bare-2014-full-transcript/',
        '2017/08/02/john-mulaney-comeback-kid-2015-full-transcript/',
        '2017/10/21/hasan-minhaj-homecoming-king-2017-full-transcript/',
        '2017/09/19/ali-wong-baby-cobra-2016-full-transcript/',
        '2017/08/03/anthony-jeselnik-thoughts-prayers-2015-full-transcript/',
        '2018/03/03/mike-birbiglia-my-girlfriends-boyfriend-2013-full-transcript/',
        '2017/08/19/joe-rogan-triggered-2016-full-transcript/']

# names of standup comedains
comedains = ['C.K.louis', 'daveChappelle', 'rickyGervais', 'boBurnham', 'billBurr', 'jimJefferies', 'johnMulaney', 'hasanMinhaj', 'aliWong', 'anthonyJeselnik', 'mikeBirbiglia', 'joeRogen']

In [15]:
# Be patient ( it takes a few minutes to run)
transcripts = [transcriptMaker(domain+u) for u in urls]

2021-01-04 18:23:01 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): scrapsfromtheloft.com:80
2021-01-04 18:23:01 [urllib3.connectionpool] DEBUG: http://scrapsfromtheloft.com:80 "GET /2017/05/06/louis-ck-oh-my-god-full-transcript/ HTTP/1.1" 200 None
2021-01-04 18:23:01 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): scrapsfromtheloft.com:80


http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/


2021-01-04 18:23:02 [urllib3.connectionpool] DEBUG: http://scrapsfromtheloft.com:80 "GET /2017/04/11/dave-chappelle-age-spin-2017-full-transcript/ HTTP/1.1" 200 None
2021-01-04 18:23:03 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): scrapsfromtheloft.com:80


http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/


2021-01-04 18:23:05 [urllib3.connectionpool] DEBUG: http://scrapsfromtheloft.com:80 "GET /2018/03/15/ricky-gervais-humanity-transcript/ HTTP/1.1" 200 None
2021-01-04 18:23:06 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): scrapsfromtheloft.com:80


...

http://scrapsfromtheloft.com/2017/08/03/anthony-jeselnik-thoughts-prayers-2015-full-transcript/


2021-01-04 18:23:33 [urllib3.connectionpool] DEBUG: http://scrapsfromtheloft.com:80 "GET /2018/03/03/mike-birbiglia-my-girlfriends-boyfriend-2013-full-transcript/ HTTP/1.1" 200 None
2021-01-04 18:23:33 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): scrapsfromtheloft.com:80


http://scrapsfromtheloft.com/2018/03/03/mike-birbiglia-my-girlfriends-boyfriend-2013-full-transcript/


2021-01-04 18:23:37 [urllib3.connectionpool] DEBUG: http://scrapsfromtheloft.com:80 "GET /2017/08/19/joe-rogan-triggered-2016-full-transcript/ HTTP/1.1" 200 None


http://scrapsfromtheloft.com/2017/08/19/joe-rogan-triggered-2016-full-transcript/


In [16]:
# Pickle files for later use ( time = 0)
# Make a new directory to hold the text files
!mkdir transcripts
for i, c in enumerate(comedains):
     with open("transcripts/" + c + ".txt", "wb") as file:
         pickle.dump(transcripts[i], file)

In [17]:
!ls

checkpoints.csv  data.csv  FinalData.csv  pictures  transcripts


In [18]:
# Loading pickled files ( time > 0 )
database = {}
for i, c in enumerate(comedains):
    with open("transcripts/" + c + ".txt", "rb") as file:
        database[c] = pickle.load(file)

In [19]:
# Double check to make sure data has been loaded properly
database.keys()

dict_keys(['C.K.louis', 'daveChappelle', 'rickyGervais', 'boBurnham', 'billBurr', 'jimJefferies', 'johnMulaney', 'hasanMinhaj', 'aliWong', 'anthonyJeselnik', 'mikeBirbiglia', 'joeRogen'])

In [20]:
# More checks
database['C.K.louis'][:2]

['Intro\nFade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily agree with you, but I appreciate very much. Well, this is a nice place. This is easily the nicest place For many miles in every direction. That’s how you compliment a building And shit on a town with one sentence. It is odd around here, as I was driving here. There doesn’t seem to be any difference Between the sidewalk and the street for pedestrians here. People just kind of walk in the middle of the road. I love traveling And seeing all the different parts of the country. I live in New York. I live in a– There’s no value to your doing that at all.',
 '“The Old Lady And The Dog”\nI live– I live in New York. I always– Like, there’s this old lady in my neighborhood, And she’s always walking her dog. She’s always just– she’s very old. She just stands there just being old, And the dog just fights gravity every day, just– The two of them, it’s re

## Part two

Another way for crawling and scraping is using softwares.
One the best ones is **octoparse.com**

# ****Refs****



https://www.sslproxies.org/

https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/

https://www.youtube.com/watch?v=ng2o98k983k

https://www.octoparse.com/blog/scrape-websites-without-being-blocked

https://www.octoparse.com/



**Thanks for your time**